In [2]:
import pandas as pd
data_pred = pd.read_csv("../raw_data/Final_Augmented_dataset_Diseases_and_Symptoms.csv")

In [3]:
data_pred.head()

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
len(data_pred.diseases.unique())
len(data_pred)

246945

In [13]:
data_pred.drop(columns = "diseases").mean().mean()*378

np.float64(5.34699693920182)

# NLP Task

### Transforming text input to required vector format

In [3]:
text = "headache, nauseous, nervousness, shaking, fever"
symptoms = ["headache", "nauseousness", "anxiety", "tremors"]

text_tokens = text.split(",")
text_tokens = [a.strip() for a in text_tokens]
text_tokens


['headache', 'nauseous', 'nervousness', 'shaking', 'fever']

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v2')
embeddings_data = model.encode(symptoms)
embeddings_input = model.encode(text_tokens)

/home/ricardasch/.pyenv/versions/3.10.6/envs/MedAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
embeddings_data[0].shape

(768,)

In [9]:
embeddings_input.shape

(5, 768)

In [10]:
import numpy as np
similiarities = np.zeros(shape = (4, 5))


In [22]:
from sentence_transformers import util

# Compute cosine similarity
for i in range(len(text_tokens)):
    cosine_scores = util.cos_sim(embeddings_input[i], embeddings_data)
    print(cosine_scores)
    print(np.argmax(cosine_scores))
    print(f"{text_tokens[i]} matches {symptoms[np.argmax(cosine_scores)]} with probability {cosine_scores.max()}")

# # Get the most similar symptoms
# top_matches = sorted(enumerate(cosine_scores[0]), key=lambda x: x[1], reverse=True)
# for idx, score in top_matches[:5]:
#     print(f"Match: {symptoms_list[idx]} (Score: {score:.4f})")


tensor([[1.0000, 0.3025, 0.2897, 0.0522]])
tensor(0)
headache matches headache with probability 1.0000001192092896
tensor([[0.3083, 0.9660, 0.4806, 0.0711]])
tensor(1)
nauseous matches nauseousness with probability 0.9660372734069824
tensor([[0.2599, 0.6796, 0.5883, 0.1451]])
tensor(1)
nervousness matches nauseousness with probability 0.6796494126319885
tensor([[0.1316, 0.1603, 0.1361, 0.1714]])
tensor(3)
shaking matches tremors with probability 0.17139729857444763
tensor([[0.1896, 0.1592, 0.1380, 0.2380]])
tensor(3)
fever matches tremors with probability 0.23804573714733124


In [ ]:
for i in len(text_tokens):
    liste = []
    for b in len(symptoms):


In [48]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randint(0,2, size=(1,8)))
df

,0,1,2,3,4,5,6,7
0,0,1,0,0,0,0,0,1


In [21]:
df.iloc[0]

0    6
1    1
2    3
3    1
4    6
5    4
6    3
7    8
Name: 0, dtype: int64

In [51]:
import requests
df = pd.DataFrame(np.random.randint(0,2, size=(1,8)))

url = 'http://localhost:8080/predict_disease'
df_dict = {"data": df.to_dict(orient="records")}
res = requests.post(url, json=df_dict)
res.json()


{'result1': 'Here is prediction 1: 3', 'result2': 'Here is prediction2: 0.375'}

In [7]:
def input_creator(text, data):
    model = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v2')
    #Load Model from pickle
    #model = load_nlp_model()

    #prepare
    data = data
    columns = list(data.columns)

    #tokenizer
    symptoms = text.split(",")
    symptoms = [a.strip() for a in symptoms]

    #embeddings
    embeddings_symptoms = model.encode(symptoms)
    embeddings_columns = model.encode(columns)

    #similarities
    similiarities = np.zeros(shape = (len(columns), len(symptoms)))

    #Setup output file
    zero_data = np.zeros(shape=(1, len(columns)))
    vector = pd.DataFrame(zero_data, columns=columns)

    # Compute cosine similarity
    for i in range(len(symptoms)):
        cosine_scores = util.cos_sim(embeddings_symptoms[i], embeddings_columns)
        print(f"{symptoms[i]} matches {columns[np.argmax(cosine_scores)]} with probability {cosine_scores.max()}")
        vector[columns[np.argmax(cosine_scores)]] = 1
    return vector

In [4]:
import pandas as pd
data_pred = pd.read_csv("../raw_data/Final_Augmented_dataset_Diseases_and_Symptoms.csv")

In [2]:
ex1 = "headache, bad fever, rash, nauseous, pressure on the eye"

In [5]:
from sentence_transformers import SentenceTransformer, util
def input_creator_med(text, data):
    model = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')
    #Load Model from pickle
    #model = load_nlp_model()

    #prepare
    data = data
    columns = list(data.columns)

    #tokenizer
    symptoms = text.split(",")
    symptoms = [a.strip() for a in symptoms]

    #embeddings
    embeddings_symptoms = model.encode(symptoms)
    embeddings_columns = model.encode(columns)

    #similarities
    similiarities = np.zeros(shape = (len(columns), len(symptoms)))

    #Setup output file
    zero_data = np.zeros(shape=(1, len(columns)))
    vector = pd.DataFrame(zero_data, columns=columns)

    # Compute cosine similarity
    for i in range(len(symptoms)):
        cosine_scores = util.cos_sim(embeddings_symptoms[i], embeddings_columns)
        print(f"{symptoms[i]} matches {columns[np.argmax(cosine_scores)]} with probability {cosine_scores.max()}")
        vector[columns[np.argmax(cosine_scores)]] = 1
    return vector

In [8]:
import numpy as np
vector1 = input_creator(ex1, data_pred)

headache matches headache with probability 1.0000001192092896
bad fever matches feeling ill with probability 0.5291414260864258
rash matches skin rash with probability 0.2823391258716583
nauseous matches nausea with probability 0.8684877157211304
pressure on the eye matches pain in eye with probability 0.7268694639205933


In [9]:
vector2 = input_creator_med(ex1, data_pred)

headache matches headache with probability 1.0000001192092896
bad fever matches fever with probability 0.8296034932136536
rash matches skin rash with probability 0.9113404154777527
nauseous matches nasal congestion with probability 0.47669023275375366
pressure on the eye matches eye strain with probability 0.7148137092590332
